Linus: generalisering, skapa regler som säger ok, h är har det uppstått ett nytt fel som vi inte upptäckt innan. skapa kod som förebygger sådana saker. tänk virusskydd som ska skydda men där en bakterie kan ändra sitt beteende och angripa ändå. alltså att skapa beredskap
för att hantera nyinkommande fel

In [23]:
import pandas as pd

# Ladda datasetet
df = pd.read_csv("transactions.csv", dtype={"receiver_account": str})


# Se till att datan är korrekt inläst
print(df.head())  # Visar första raderna av datan


                         transaction_id            timestamp     amount  \
0  f2f3b0fc-b7d7-4d85-b682-cbf07ef77c1a  2025-03-06 12:04:00    7746.03   
1  feccbd1e-7752-4452-a44a-901510252561  2025-03-31 01:37:00   45193.04   
2  bc6a480c-e2e1-434d-830c-1cda3f913038  2025-04-29 22:58:00   33029.71   
3  b832aa2c-77f3-4c4f-9697-9776b8d74abb  2025-02-27 14:52:00  35994 .61   
4  0d32c6ef-508e-4f5d-bc64-341a0a9f498d  2025-01-03 22:29:00   32229.73   

  currency            sender_account          receiver_account sender_country  \
0      SEK  SE8902EWFT80524424320740  SE8902ZUIH32054423564254         Sweden   
1      SEK  SE8902GTSI58472323675409  SE8902ERNA19462716907689         Sweden   
2      SEK  SE8902SEZT63377163386089  SE8902LOLQ30361773270969         Sweden   
3      SEK  SE8902MGEE43594824630478  SE8902XIVG18316797897983         Sweden   
4      SEK  SE8902SZWO66860675966705  SE8902SUUS97732018069804         Sweden   

  sender_municipality receiver_country receiver_municipality t

C:\Users\milan\AppData\Local\Temp\ipykernel_61156\795136958.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("transactions.csv", dtype={"receiver_account": str})


In [16]:
print(df.iloc[:, 2].apply(type).value_counts())  # Kontrollera vilka typer som finns i kolumn (2)


amount
<class 'str'>      65536
<class 'float'>    34464
Name: count, dtype: int64


In [17]:
print(df.iloc[:, 2].apply(type).value_counts())  # Lista datatyper och antal rader


amount
<class 'str'>      65536
<class 'float'>    34464
Name: count, dtype: int64


In [18]:
print(df["amount"].loc[df["amount"].apply(lambda x: isinstance(x, str))].sample(20))  # Slumpmässigt urval av 20 strängvärden


63333    30943.23
614      29990.41
58650     7288.47
64968    19698.94
57619     3967.94
42194    21205.46
62758    47450.85
33352    18722.17
4127     13153.96
59792    47875.31
36780    15269.49
1476     28149.52
51665     43314.3
6595     29839.75
53576    25320.57
51024     13681.2
12502     3174.34
16542    40039.53
20087     4244.21
27026    39584.67
Name: amount, dtype: object


In [19]:
df["amount"] = pd.to_numeric(df["amount"], errors="coerce")


In [88]:
df.dtypes

transaction_id                   object
timestamp                datetime64[ns]
amount                          float64
currency                         object
sender_account                   object
receiver_account                 object
sender_country                   object
sender_municipality              object
receiver_country                 object
receiver_municipality            object
transaction_type                 object
notes                            object
suspicious                         bool
foreign_account                    bool
internal_transaction               bool
dtype: object

In [1]:
from pydantic import BaseModel, Field
from datetime import datetime
from typing import Optional

class Transaction(BaseModel):
    transaction_id: str
    amount: float = Field(gt=0, description="Transaktionsbelopp måste vara positivt")
    timestamp: datetime
    currency: str
    sender_account: str
    receiver_account: str
    sender_country: str
    sender_municipality: str
    receiver_country: str
    receiver_municipality: str
    transaction_type: str
    notes: Optional[str]  # Kan vara tomt



In [44]:
data = {
    "transaction_id": "12345",
    "amount": 100,
    "timestamp": "2025-05-22T08:44:00",
    "currency": "EUR",
    "sender_account": "ABC123",
    "receiver_account": "XYZ789",
    "sender_country": "Sweden",
    "sender_municipality": "Stockholm",
    "receiver_country": "Germany",
    "receiver_municipality": "Berlin",
    "transaction_type": "purchase",
    "notes": None
}


try:
    transaction = Transaction(**data)
    print(transaction)
except Exception as e:
    print(f"Fel vid validering: {e}")


transaction_id='12345' amount=100.0 timestamp=datetime.datetime(2025, 5, 22, 8, 44) currency='EUR' sender_account='ABC123' receiver_account='XYZ789' sender_country='Sweden' sender_municipality='Stockholm' receiver_country='Germany' receiver_municipality='Berlin' transaction_type='purchase' notes=None


In [2]:
valid_transactions = []
invalid_transactions = []

for index, row in df.iterrows():
    try:
        transaction = Transaction(**row.to_dict())
        valid_transactions.append(transaction)
    except Exception as e:
        invalid_transactions.append((index, str(e)))

print(f"Antal giltiga transaktioner: {len(valid_transactions)}")
print(f"Antal ogiltiga transaktioner: {len(invalid_transactions)}")


NameError: name 'df' is not defined

In [91]:
from collections import Counter

# Räkna de vanligaste felorsakerna
error_counts = Counter([error[1] for error in invalid_transactions])
print(error_counts.most_common(1))  # Visa de 10 vanligaste felorsakerna


[]


In [39]:
df.fillna("", inplace=True)  # Byter ut alla NaN mot tomma strängar


In [40]:
df["notes"] = df["notes"].fillna("").astype(str)  # Ersätt NaN med tomma strängar och tvinga `str`-format


In [45]:
for index, error in invalid_transactions:
    print(f"Rad {index}: {error}")


Rad 3: 1 validation error for Transaction
amount
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='35994 .61', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/float_parsing
Rad 48: 1 validation error for Transaction
amount
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='33533 81', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/float_parsing
Rad 372: 1 validation error for Transaction
timestamp
  Input should be a valid datetime or date, invalid date separator, expected `-` [type=datetime_from_date_parsing, input_value='20250125 04:48:00', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/datetime_from_date_parsing
Rad 374: 1 validation error for Transaction
timestamp
  Input should be a valid datetime or date, invalid character in year [type=datetime_from_date_parsing, in

In [46]:
df["amount"] = df["amount"].astype(str).str.replace(" ", "").astype(float)


In [51]:
print(df["timestamp"].apply(type).value_counts())  # Kontrollera datatyper i timestamp
print(df["timestamp"].head(20))  # Visa exempel på timestamps


timestamp
<class 'str'>    100000
Name: count, dtype: int64
0     2025-03-06 12:04:00
1     2025-03-31 01:37:00
2     2025-04-29 22:58:00
3     2025-02-27 14:52:00
4     2025-01-03 22:29:00
5     2025-01-13 04:03:00
6     2025-01-25 03:01:00
7     2025-04-07 16:48:00
8     2025-04-08 09:16:00
9     2025-05-17 20:20:00
10    2025-01-23 17:07:00
11    2025-01-24 05:54:00
12    2025-01-22 17:45:00
13    2025-03-01 01:21:00
14    2025-01-23 21:47:00
15    2025-05-17 02:07:00
16    2025-02-23 08:45:00
17    2025-05-08 18:22:00
18    2025-02-19 12:34:00
19    2025-03-11 18:00:00
Name: timestamp, dtype: object


In [52]:
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")


In [53]:
invalid_timestamps = df.loc[df["timestamp"].isna()]  # Hitta NaT-värden
print(invalid_timestamps)


                            transaction_id timestamp    amount currency  \
372   e13b1664-c549-4f92-895a-1f941dd6d82e       NaT  12881.63      SEK   
374   7bfcf599-e449-4e63-a3f9-05631511e58a       NaT  20223.17      SEK   
432   b75a64f1-d111-4261-9df0-53f2c38e2828       NaT  45069.28      SEK   
566   bc91f1d3-e0b0-471b-844f-7e14941d3236       NaT   9908.07      SEK   
658   e4ec0cfc-55e8-4ca1-b3f4-0948a1b8e997       NaT   3080.18      SEK   
661   13638365-972b-4fd8-9518-f27557a7e1cd       NaT  34157.12      SEK   
761   fda4c622-1044-49aa-bc41-b44bdd50bc9c       NaT   2192.15      SEK   
814   bc23bd09-5b6a-4ebe-95ae-7dad0cde315a       NaT  21899.83      SEK   
817   9687e5ce-f8e0-4afa-ac0c-54d8c6ef457f       NaT  36969.56      SEK   
821   aee31e4c-e8c1-4fa5-b8bf-53815ab51b55       NaT  39794.00      SEK   
866   f46345d0-c1f8-46d7-ae78-47c4a6b52fee       NaT   9573.92      SEK   
876   2f48c8d5-cad7-48b0-8f8c-2f1324911dfd       NaT  34373.92      SEK   
884   776cbccf-2d92-4e83-

In [54]:
df["timestamp"] = df["timestamp"].fillna(pd.Timestamp("1900-01-01 00:00:00"))


implementering av regler

In [84]:
import datetime

def is_suspicious(transaction):
    now = datetime.datetime.now()
    return (
        transaction["amount"] <= 0 or
        transaction["timestamp"] > now or
        transaction["sender_account"] == transaction["receiver_account"]
    )



In [85]:
def is_foreign_account(sender_country):
    return sender_country != "Sweden"  # Jämför direkt mot landnamn istället för landkod




In [86]:
def is_internal_transaction(sender_country, receiver_country):
    return sender_country == "Sweden" and receiver_country == "Sweden"


In [78]:
df["suspicious"] = df.apply(is_suspicious, axis=1)
df["foreign_account"] = df["sender_country"].apply(lambda x: x != "Sweden")
df["internal_transaction"] = df.apply(lambda x: is_internal_transaction(x["sender_country"], x["receiver_country"]), axis=1)


In [79]:
print(df["foreign_account"].value_counts())


foreign_account
False    80473
True     19527
Name: count, dtype: int64


In [80]:
print(df[df["foreign_account"] == True].head(10))


                            transaction_id           timestamp    amount  \
188   709e9718-7e51-4f56-9b55-d44615eace43 2025-04-27 17:50:00  24156.67   
296   eaebcdd9-5ba6-4966-9bd0-3bac7d6ba12c 2025-02-11 15:01:00   5020.92   
496   519cd1ac-0199-4e40-a313-9e836bc07b5f 2025-03-15 21:16:00  36157.41   
633   a05cf133-2531-4584-bd76-9968e77f4fe7 2025-03-30 18:46:00  14434.71   
661   13638365-972b-4fd8-9518-f27557a7e1cd 1900-01-01 00:00:00  34157.12   
864   ad008209-be9f-42df-9e9a-8f1d2f62ace1 2025-03-01 05:13:00  21730.91   
941   53e87c84-5146-4511-a890-69d427dceb69 2025-03-14 23:11:00   5673.19   
1606  5cd62eb6-1a23-4025-bee4-6814c667f0bb 2026-01-24 01:12:00  27159.91   
1690  13b2ff5d-0e14-406e-aca7-12c67c4f08c2 2025-01-20 03:54:00  35498.21   
2285  ef21a9a7-1c36-4dc9-85b6-8d5be15c41a4 2025-04-10 03:50:00  29344.24   

     currency            sender_account          receiver_account  \
188       SEK  SE8902IGFR09709190696891  SE8902QORV15540178275192   
296       SEK  SE8902PHZM

In [81]:
print(df[df["foreign_account"] == True][["sender_country"]].head(10))


     sender_country
188                
296                
496                
633                
661                
864                
941                
1606               
1690               
2285               


In [82]:
print(df["sender_country"].unique())


['Sweden' '' 'Denmark' 'Zimbabwe' 'Estonia' 'Germany' 'Ecuador' 'Lebanon'
 'Norway' 'Botswana' 'Reunion' 'Cambodia' 'Senegal' 'Nigeria'
 'Liechtenstein' 'Isle of Man' 'Saudi Arabia' 'Cuba' 'Monaco'
 'Libyan Arab Jamahiriya' 'Georgia' 'Dominican Republic' 'Montserrat'
 'Bosnia and Herzegovina' 'Cook Islands' 'Venezuela'
 'Holy See (Vatican City State)' 'Finland' 'Bulgaria'
 'Turks and Caicos Islands' 'Suriname' 'United States Virgin Islands'
 'Iran' 'Slovakia (Slovak Republic)' 'Island' 'Haiti' 'Luxembourg'
 'Saint Martin' 'Angola' 'Guatemala' "Cote d'Ivoire" 'Oman'
 'Christmas Island' 'Singapore' 'Belarus' 'Lithuania'
 'Antarctica (the territory South of 60 deg S)' 'Martinique' 'Kenya'
 'Turkmenistan' 'Fiji' 'Azerbaijan' 'Burkina Faso' 'United Arab Emirates'
 'Andorra' 'Poland' 'Eritrea' 'Niue' 'Togo' 'Malaysia' 'Liberia' 'Ireland'
 'Ethiopia' 'Cape Verde' 'Antigua and Barbuda' 'Myanmar' 'Iceland'
 'Bahamas' 'Svalbard & Jan Mayen Islands' 'Belize' 'Burundi' 'Benin'
 'Croatia' 'Namibia'

In [83]:
print(df[df["sender_country"] == "Sweden"]["foreign_account"].value_counts())


foreign_account
False    80473
Name: count, dtype: int64


In [60]:
%whos


Variable                        Type              Data/Info
-----------------------------------------------------------
BaseModel                       ModelMetaclass    <class 'pydantic.main.BaseModel'>
Counter                         type              <class 'collections.Counter'>
Field                           function          <function Field at 0x000002689A094C20>
Optional                        _SpecialForm      typing.Optional
Transaction                     ModelMetaclass    <class '__main__.Transaction'>
data                            dict              n=12
datetime                        module            <module 'datetime' from '<...>oject\\Lib\\datetime.py'>
df                              DataFrame                                  <...>100000 rows x 14 columns]
error                           str               1 validation error for Tr<...>atetime_from_date_parsing
error_counts                    Counter           Counter({'1 validation er<...>/2.10/v/string_type': 1})
i

Regler för små transaktioner

In [92]:
df["small_transactions"] = (df["amount"] < 100)  # Transaktioner under 100 är små transaktioner
df["transaction_hour"] = df["timestamp"].dt.hour  # Extrahera timme


In [4]:
df["transaction_date"] = df["timestamp"].dt.date  # Extrahera datum
small_tx_counts = df[df["small_transactions"]].groupby(["sender_account", "amount", "transaction_date"]).size()
print(small_tx_counts)


NameError: name 'df' is not defined

In [95]:
df["small_tx_count"] = df.groupby(["sender_account", "transaction_date"])["amount"].transform(lambda x: (x < 100).sum())
df["flag_frequent_small_tx"] = df["small_tx_count"] > 5


In [96]:
flagged_accounts = df[df["flag_frequent_small_tx"]]
print(flagged_accounts[["sender_account", "small_tx_count", "transaction_date"]])


Empty DataFrame
Columns: [sender_account, small_tx_count, transaction_date]
Index: []


In [97]:
print(df["small_tx_count"].describe())


count    100000.00000
mean          0.00333
std           0.05761
min           0.00000
25%           0.00000
50%           0.00000
75%           0.00000
max           1.00000
Name: small_tx_count, dtype: float64


Regeln för små transaktioner

In [98]:
# Extrahera transaktionsdatum
df["transaction_date"] = df["timestamp"].dt.date

# Beräkna antalet små transaktioner per konto och dag
df["small_tx_count"] = df.groupby(["sender_account", "transaction_date"])["amount"].transform(lambda x: (x < 100).sum())

# Flagga konton med mer än 5 små transaktioner på en dag
df["flag_frequent_small_tx"] = df["small_tx_count"] > 5


Regler för förekomsten av ordet "gift" i kolumnen notes

In [102]:
df["flag_contains_gift"] = df["notes"].str.contains(r"\bgift\b", case=False, na=False)


In [103]:
print(df[df["flag_contains_gift"]][["transaction_id", "notes"]])


                             transaction_id                  notes
15     095cf949-0f07-4119-8dfc-0cc10069de0b  Gift to family member
33     51c7adf0-9c6a-4dd9-a788-e3211b934571  Gift to family member
40     f2f5437f-d87a-4534-a2bc-37c247f182ed  Gift to family member
78     c255762b-897b-4c41-9777-0e5e09d20db8  Gift to family member
88     596cda27-e5da-4880-8c4c-908a1a0841fa  Gift to family member
...                                     ...                    ...
99958  f6f246c2-5dbd-4357-8d9f-d4652c3721d9  Gift to family member
99976  b97dbc15-63ed-4f5b-8a05-facbb3745af2  Gift to family member
99987  49f879a9-d54c-4567-a88e-13d02f0777bf  Gift to family member
99995  ea7e0cb2-fadb-496c-962b-7e54a5cfa1bc  Gift to family member
99997  23574238-9739-4af4-a823-5b699812db4d  Gift to family member

[6398 rows x 2 columns]


Regler för avvikande transaktioner

In [119]:
df["avg_amount"] = df.groupby("sender_account")["amount"].transform("mean") #Varje rad får sitt kontospecifika snitt

# Flagga stora transaktioner: Mer än 2x snittet (100% högre)
df["flag_large_tx"] = df["amount"] > (df["avg_amount"] * 2)

# Flagga små transaktioner: Mindre än 10% av snittet (90% lägre)
df["flag_small_tx"] = df["amount"] < (df["avg_amount"] * 0.1)


test bara

In [120]:
print(df["flag_large_tx"])

0    False
1    False
2    False
3    False
4    False
5    False
6    False
Name: flag_large_tx, dtype: bool


Generalisering av kod för att upptäcka anomali i transaktioner



In [113]:
from sklearn.ensemble import IsolationForest
import pandas as pd

# Skapa DataFrame (exempeldata)
df = pd.DataFrame({
    "transaction_id": [1, 2, 3, 4, 5, 6, 7],
    "amount": [100, 200, 150, 5000, 180, 190, 50000],  # Ovanligt höga belopp
    "sender_account": ["A", "A", "B", "C", "B", "B", "C"]
})

# Träningsdata: Vi analyserar transaktionsbeloppet
model = IsolationForest(contamination=0.05, random_state=42)  # Anpassar sig till ca 5% avvikande transaktioner
df["flag_anomaly"] = model.fit_predict(df[["amount"]])

# Markera anomalier (de får -1 i modellen)
df["flag_anomaly"] = df["flag_anomaly"] == -1

print(df[["transaction_id", "amount", "flag_anomaly"]])


   transaction_id  amount  flag_anomaly
0               1     100         False
1               2     200         False
2               3     150         False
3               4    5000         False
4               5     180         False
5               6     190         False
6               7   50000          True


In [114]:
# Räkna antal anomalier (flag_anomaly = True)
num_anomalies = df["flag_anomaly"].sum()

print(f"Antal flaggade anomalier: {num_anomalies}")


Antal flaggade anomalier: 1
